# Homework 1

#### EE-556 Mathematics of Data - Fall 2024


In this homework, we consider a multiclass classification task modeled by multinomial (softmax) logistic regression. Your goal will be to analyze the estimator and its properties (convexity, existence/uniqueness), and to derive gradients/Hessians and smoothness bounds. The first part consists of theoretical questions only.


<div class="alert alert-info">
  ℹ️ <strong>Information on group based work:</strong>
</div>

- You are to deliver only 1 notebook per group.
- Asking assistance beyond your group is ok, but answers should be individual to the group.
- In the event that there was <span style="color: red;">disproportional work done</span> by different group members, let the TAs know.
- Only one member of the group is allowed to use AI. We will require sharing the conversation history with the AI in the form of a public link. If you use multiple conversations across the same or multiple tools please share all of them. Name the person in your group who is allowed to use AI. We encourage you to use the AI to help you understand the material, but we ask you to write the code and theory solutions by yourself.

<div style="border: 1px solid #f00; background-color: #fdd; padding: 10px; border-radius: 5px;">
  ⚠️ Do not forget: Write who are the people in your group as well as their respective SCIPER numbers
</div>


Person 1 **Name**: Chenyue Fang || Person 1 **SCIPER**: 413183


Person 2 **Name**:Xingyu Pan || Person 2 **SCIPER**: 369420


Person 3 **Name**: || Person 3 **SCIPER**:


<div style="border: 1px solid #0a0; background-color: #dfd; padding: 10px; border-radius: 5px;">
  📓 Feedback on AI use: Please use the following cell to provide feedback on the AI use in this notebook.
  
  For example, how useful were the tools to you? Which tools did you use? Did you feel like they helped you understand the material better?
</div

## 1. Multiclass Softmax Logistic Regression - 15 Points


We now model multiclass classification with classes $c \in \{1,\dots,C\}$. For each sample $(\mathbf{a}_i, b_i)$ with $\mathbf{a}_i \in \mathbb{R}^p$ and $b_i \in \{1,\dots,C\}$, let $\mathbf{X} = [\mathbf{x}_1,\dots,\mathbf{x}_C] \in \mathbb{R}^{p\times C}$ be the class weight matrix. The softmax model defines

$$
\mathbb{P}(b_i = c \mid \mathbf{a}_i) = \frac{\exp(\mathbf{a}_i^\top \mathbf{x}_c)}{\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)}.
$$

Assume i.i.d. samples $\{(\mathbf{a}_i,b_i)\}_{i=1}^n$. Our goal is to estimate $\mathbf{X}$ by maximum likelihood (and later with an $\ell_2$ regularizer).


__(a)__ (1 point) Show that the negative log-likelihood $f$ can be written as:

$$
\begin{aligned}
 f(\mathbf{X})
 &= - \log \mathbb{P}(b_1,\dots,b_n\mid \mathbf{a}_1,\dots,\mathbf{a}_n)\\
 &= \sum_{i=1}^n \left[ -\mathbf{a}_i^\top \mathbf{x}_{b_i} + \log \sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k) \right].
\end{aligned}
$$



Proof:

Because samples $\{(a_i,b_i)\}^n_{i=1}$ i.i.d, 
$$
\begin{aligned}
\mathbb{P}(b_1,\cdots,b_n|\mathbf{a_1,\cdots,a_n})
&=\prod_{i=1}^n\mathbb{P}(b_i|\mathbf{a}_i)\\
&=\prod_{i=1}^n\frac{\exp(\mathbf{a}_i^\top\mathbf{x}_{b_i})}{\sum_{k=1}^C\exp(\mathbf{a}_i^\top\mathbf{x}_k)}.
\end{aligned}
$$
Applying the negative log-likelihood, we can get:
$$
\begin{aligned}
f(\mathbf{X})
&= - \log \mathbb{P}(b_1,\dots,b_n\mid \mathbf{a}_1,\dots,\mathbf{a}_n)\\
&=-\log(\prod_{i=1}^n\frac{\exp(\mathbf{a}_i^\top\mathbf{x}_{b_i})}{\sum_{k=1}^C\exp(\mathbf{a}_i^\top\mathbf{x}_k)})\\
&=-\sum_{i=1}^n \left[\mathbf{a}_i^\top\mathbf{x}_{b_i}-\log\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)\right] \\
&=\sum_{i=1}^n \left[-\mathbf{a}_i^\top\mathbf{x}_{b_i}+\log\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)\right].
\end{aligned}
$$

__(b)__ (2 points) Show that $\mathbf{u} \mapsto \log\!\left(\sum_{k=1}^C e^{u_k}\right)$ is convex on $\mathbb{R}^C$. Then, show that $f(\mathbf{X})$ is convex.


Hint: use Jensen's inequality.

Proof:

First, we will show that $\mathbf{u} \mapsto \log\!\left(\sum_{k=1}^C e^{u_k}\right)$ is convex on $\mathbb{R}^C$ by proving $\forall\,\mathbf{u}, \mathbf{w} \in \mathbb{R}^C$ and $\alpha\in[0,1]$:
$$
\alpha\log\!\left(\sum_{k=1}^C e^{u_k}\right)+(1-\alpha)\log\!\left(\sum_{j=1}^C e^{w_j}\right)\geq\log\!\left(\sum_k^C e^{\alpha u_k+(1-\alpha)w_k}\right)
$$

We set $g(x)=\exp(x), \,h(x)=\log(x)$, it is easy to derive that $\nabla^2g(x)>0$ and $\nabla^2h(x)<0$ for any $x \in \mathbf{R}$, therfore $g(x)$ is a convex function and $h(x)$ is a concave function.

According to Jensen's inequality, $e^{\alpha u_k+(1-\alpha) w_k}\leq\alpha x^{u_k}+(1-\alpha)e^{w_k}$ for any integer $k \in\{1,2,\cdots,C\}$. Therefore, we can get
$$
\begin{aligned}
\log\!\left(\sum_k^C e^{\alpha u_k+(1-\alpha)w_k}\right)
&\leq\log\!\left(\sum_k^C \left[\alpha e^{u_k}+(1-\alpha)e^{w_k}\right]\right)\\
&=\log\!\left(\alpha S_1+(1-\alpha)S_2\right),
\end{aligned}
$$
in which $S_1=\sum_k^C \alpha e^{u_k}$ and $S_2=\sum_k^C \alpha e^{w_k}$.

And also according to Jensen's inequality, 
$$
\begin{aligned}
\log\!\left(\alpha S_1+(1-\alpha)S_2\right)
&\leq\alpha\log\!\left(S_1\right)+(1-\alpha)\log\!\left(S_2\right)\\
&=\alpha\log\!\left(\sum_{k=1}^C e^{u_k}\right)+(1-\alpha)\log\!\left(\sum_{j=1}^C e^{w_j}\right).
\end{aligned}
$$

Next, I are going to prove $f(\mathbf{X})$ is convex by proving $-\mathbf{a}_i^\top\mathbf{x}_{b_i}+\log\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)$ is convex for any integer $i\in\{1,2,\cdots,C\}$, because the sum of convex functions is still a convex function.

We just pick an arbitrary $i\in\{1,2,\cdots,C\}$. Since we have already proven that $\mathbf{u} \mapsto \log\!\left(\sum_{k=1}^C e^{u_k}\right)$ is convex on $\mathbb{R}^C$, so $\log\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)$ is convex. Combining the fact that linear term won't effect the function's convexity, we show that the convexity of $-\mathbf{a}_i^\top\mathbf{x}_{b_i}+\log\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)$ is same as $\log\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)$, which means it is convex.


You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator
$$
\mathbf{X}^\star_{ML} = \arg\min_{\mathbf{X} \in \mathbb{R}^{p\times C}} f(\mathbf{X})
$$

is a global minimum. But does the minimum always exist? We will ponder this question in the following three points.


__(c)__ (1 point) Explain the difference between infima and minima. Give an example of a convex function on $\mathbb{R}$ that does not attain its infimum.


The infimum is the lower bound of the function's values, which it may or may not actually reach.

The minimum is the smallest value a function actually reaches.

Example:

$f(x)=\exp(x)$ is a convex function defined on $\mathbb{R}$ and does not attain its infimum 0.

__(d)__ (1 point) Assume there exists $\mathbf{X}_0 \in \mathbb{R}^{p\times C}$ such that for all $i$,
$$
\mathbf{a}_i^\top \mathbf{x}_{0, b_i} - \max_{k \neq b_i} \mathbf{a}_i^\top \mathbf{x}_{0,k} > 0.
$$
This is called one-versus-all complete separation in multiclass settings. Give a geometric interpretation (e.g., for $p=2$) and explain why the name is appropriate.


The condition can be rewritten as:
$$
\begin{aligned}
\mathbf{a}_i^\top(\mathbf{x}_{0,b_i}-\mathbf{x}_{0,k})>0, \,\forall k \neq b_i.
\end{aligned}
$$
The equation $\mathbf{a}_i^\top(\mathbf{x}_{0,b_i}-\mathbf{x}_{0,k})=0$ defines a line that passes through the origin and is perpendicular to the vector ($\mathbf{x}_{0,b_i}-\mathbf{x}_{0,k}$). The condition says that for any data ponit $\mathbf{a}_i$, it will lie on the positive side of the plane defined between its true class $b_i$ and any other contrastive class $k$. 

From this, you should see that it is likely that some datasets satisfy the complete separation assumption. Unfortunately, as you will show next, this can become an obstacle.


__(e)__ (1 point) In a one-versus-all complete separation setting (as in (d)), prove that $f$ does not attain its minimum. Hint: consider $f(\alpha \mathbf{X}_0)$ as $\alpha \to +\infty$ and compare it to $f(\mathbf{X}_0)$.


Proof:

From part (a),
$$
\begin{aligned}
f(\alpha\mathbf{X}_0)
&=\sum_{i=1}^n \left[ -\mathbf{a}_i^\top (\alpha\mathbf{x}_{0,b_i}) + \log \sum_{k=1}^C \exp(\mathbf{a}_i^\top (\alpha\mathbf{x}_{0,k})) \right] \\[6pt]
&=\sum_{i=1}^n \left[ -\alpha(\mathbf{a}_i^\top \mathbf{x}_{0,b_i}) + \log \sum_{k=1}^C \exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,k}\big) \right] \\[6pt]
&=\sum_{i=1}^n \left[ -\alpha(\mathbf{a}_i^\top \mathbf{x}_{0,b_i}) + 
\log\!\left( \exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,b_i}\big)
\sum_{k=1}^C 
\frac{\exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,k}\big)}
     {\exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,b_i}\big)} 
\right) \right] \\[6pt]
&=\sum_{i=1}^n \left\{ 
-\alpha(\mathbf{a}_i^\top \mathbf{x}_{0,b_i}) 
+ \log\!\left( 
\exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,b_i}\big)
\left[ 1 + \sum_{k\neq b_i} 
\frac{\exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,k}\big)}
     {\exp\!\big(\alpha\,\mathbf{a}_i^\top \mathbf{x}_{0,b_i}\big)} 
\right]
\right)
\right\} \\[6pt]
&=\sum_{i=1}^n 
\log\!\left[
1 + \sum_{k\neq b_i} 
\exp\!\big(\alpha\,\mathbf{a}_i^\top (\mathbf{x}_{0,k}-\mathbf{x}_{0,b_i})\big)
\right].
\end{aligned}
$$

By the definition of complete separation, the term 
$\mathbf{a}_i^\top(\mathbf{x}_{0,k}-\mathbf{x}_{0,b_i}) < 0$ 
for all $i$ and all $k \neq b_i$. Therefore, as $\alpha \to +\infty$, 
$f(\alpha \mathbf{X}_0) \to 0$, which means the infimum of $f$ is 0.  
Considering the exponential function is strictly positive, 
$f(\alpha \mathbf{X}_0) > 0$. Therefore, $f$ does not attain its minimum.

We resolve this issue by adding a regularizer. Consider the regularized function

$$
 f_\mu(\mathbf{X}) = f(\mathbf{X}) + \frac{\mu}{2} \|\mathbf{X}\|_F^2, \quad \mu > 0.
$$

__(f)__ (1 point) Show that the gradient with respect to $\mathbf{X}$ of $f_\mu$ can be expressed as
$$
 \nabla_{\mathbf{X}} f_\mu(\mathbf{X}) = \sum_{i=1}^n \big( \mathbf{p}_i - \mathbf{e}_{b_i} \big) \mathbf{a}_i^\top + \mu \mathbf{X},\tag{1}
$$
where $\mathbf{e}_{b_i} \in \mathbb{R}^C$ is the [one-hot vector](https://en.wikipedia.org/wiki/One-hot) for class $b_i$, $\mathbf{p}_i \in \mathbb{R}^C$ has entries $p_{i,c} = \mathbb{P}(b_i=c\mid \mathbf{a}_i)$ under the softmax model, and $(\mathbf{p}_i - \mathbf{e}_{b_i})\mathbf{a}_i^\top$ denotes the outer product.



Proof:

First, we figure out the gradient of the Frobenius norm, 
$$\nabla_{\mathbf{X}}\|\mathbf{X}\|_F^2=\left(\frac{\|\mathbf{X}\|_F^2}{dx_{ij}}\right)_{i,j}=\left(\frac{dx_{i,j}^2}{dx_{ij}}\right)_{i,j}=2\mathbf{X}.$$

Then, we will compute $\nabla_{\mathbf{X}} f(\mathbf{X})$ by taking the partial derivative of a single element $\mathbf{X}_{rc}$:
$$
\begin{aligned}
\frac{\partial f}{\partial x_{rc}}
&= \frac{\partial}{\partial x_{rc}}\sum_{i=1}^n \left[ -\mathbf{a}_i^\top \mathbf{x}_{b_i} + \log \sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k) \right]\\
&= \sum_{i=1}^n \left[ -\mathbb{I}_{c=b_i}a_{ir}  + \frac{\exp(\mathbf{a}_i^\top \mathbf{x}_{c})}{\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)}a_{ir} \right]\\
&= \sum_{i=1}^n (p_{i,c}-\mathbb{I}_{c=b_i})a_{ir} \\
\end{aligned}
$$
Therefore, we can get $\nabla_{\mathbf{X}} f(\mathbf{X})$:
$$
\begin{aligned}
\nabla_{\mathbf{X}} f(\mathbf{X})
&=\left(\frac{\partial f}{\partial x_{rc}} \right)_{rc}\\
&=\sum_{i=1}^n\mathbf{a}_i(\mathbf{p}_i-\mathbf{e}_{b_i})^\top
\end{aligned}
$$
Finally, we show that:
$$
\nabla_{\mathbf{X}} f_\mu(\mathbf{X}) = \sum_{i=1}^n\mathbf{a}_i(\mathbf{p}_i-\mathbf{e}_{b_i})^\top + \mu \mathbf{X}
$$

__(g)__ (1 point) Show that the Hessian of $f_\mu$ can be written as
$$
 \nabla^2 f_\mu(\mathbf{X}) = \sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big) + \mu \mathbf{I},\tag{2}
$$
where $\otimes$ is the Kronecker product, and $\operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top$ is the softmax Jacobian, which is positive semidefinite.


Proof:

It is obvious that $\nabla\mathbf{X}=\mathbf{I}$. So, we only need to prove that $\nabla\sum_{i=1}^n\mathbf{a}_i(\mathbf{p}_i-\mathbf{e}_{b_i})^\top=\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$.
To prove the equation, we consider to calculate $\nabla\sum_{i=1}^n\mathbf{a}_i(\mathbf{p}_i-\mathbf{e}_{b_i})^\top$:
$$
\nabla\sum_{i=1}^n\mathbf{a}_i(\mathbf{p}_i-\mathbf{e}_{b_i})^\top=\sum_{i=1}^n\nabla\mathbf{a}_i\mathbf{p}_i^\top.
$$
To compute the Hessian of $\mathbf{a}_i\mathbf{p}_i^\top$ with respect to $\mathbf{X}$, we must use the chain rule on
$$
\mathbf{p}_i = \text{softmax}(\mathbf{z}_i) \quad \text{where} \quad \mathbf{z}_i = X^\top\mathbf{a}_i.
$$
First, we compute $\frac{\partial }{\partial \mathbf{X}}\mathbf{z}_i$:
$$\frac{\partial \mathbf{z}_{is}}{\partial x_{rc}}= \frac{\partial}{\partial x_{rc}} \left( \sum_{l=1}^p x_{ls} \mathbf{a}_{il} \right) = \sum_{l=1}^p \delta_{lr}\delta_{sc} \mathbf{a}_{il} = \mathbf{a}_{ir}\delta_{sc}.$$
Then, by applying the chain rule, we can show that:
$$\frac{\partial p_{iv}}{\partial x_{rc}} = \sum_{s=1}^C \frac{\partial p_{iv}}{\partial \mathbf{z}_{is}} (a_{ir}\delta_{sc}) = \frac{\partial p_{iv}}{\partial z_{ic}} a_{ir}.$$
Combining them together, we get:
$$\left(\frac{\partial (\mathbf{a}_i\mathbf{p}_i^\top)}{\partial x_{rc}}\right)_{uv} = \mathbf{a}_{iu} \left( \frac{\partial p_{iv}}{\partial \mathbf{z}_{ic}} \mathbf{a}_{ir} \right) = (\mathbf{a}_{iu}\mathbf{a}_{ir}) \left( \frac{\partial p_{iv}}{\partial \mathbf{z}_{ic}} \right).$$
According to the problem information:
$$
J_{\mathbf{p}_i} = \frac{\partial \mathbf{p}_i}{\partial \mathbf{z}_i^\top} = \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i \mathbf{p}_i^\top.
$$
From the equation above, we can observe that:
$$
\sum_{i=1}^n\nabla\left(\mathbf{a}_i\mathbf{p}_i^\top\right)=\sum_{i=1}^n\mathbf{a}_i\mathbf{a}_i^\top\otimes\left[\operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top\right].
$$
Therefore, $ \nabla^2 f_\mu(\mathbf{X}) = \sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big) + \mu \mathbf{I}$.

__(h)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex.


Proof:

To show that $f_\mu$ is $\mu$-strongly convex, we need to prove that $\nabla^2f_\mu\succeq\mu \mathbf{I}$, in other words $\sum_{i=1}^n (\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$ is a semi-positive definite matrix.

Since the sum of semi-positive definite matrices is a semi-positive definite matrix, we will show that for every $i \in \{1, \cdots, n\}$ such that $(\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$ is a semi-positive definite matrix.

By the definition of Kronecker product, if a matrix $A$ has eigenvalues $\lambda_1, \cdots, \lambda_k$ and another matrix $B$ has eigenvalues $\mu_1, \cdots, \mu_l$, then the eigenvalues of $A\otimes B$ are $\{\lambda_i\mu_j\}_{i=1,\cdots,k,\, j=1,\dots, l}$. Therefore, the Kronecker product of two semi-positive definite matrices is still a semi-positive definite matrix.

First, we prove $\mathbf{a}_i\mathbf{a}_i^\top$ is a semi-positive definite matrix:
$$
\mathbf{x}^\top\mathbf{a}_i\mathbf{a}_i^\top \mathbf{x} =(\mathbf{a}_i^\top \mathbf{x})^\top(\mathbf{a}_i^\top \mathbf{x})=\|\mathbf{a}_i^\top \mathbf{x}\|^2\ge 0,\quad \forall\, \mathbf{x}\in\mathbb{R}^p,
$$
which shows that $\mathbf{a}_i\mathbf{a}_i^\top$ is a semi-positive definite matrix.

Then, we prove $\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$ is a semi-positive definite matrix:
$$
\begin{aligned}
\mathbf{x}^\top\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)\mathbf{x} 
&=\sum_{j=1}^p p_{ij}x_j^2 - \Big(\sum_{k=1}^p p_{ik}x_k\Big)^2 \\[4pt]
&=\mathbb{E}[X^2] - (\mathbb{E}[X])^2 \\[4pt]
&=\operatorname{Var}(X)\ge 0,
\end{aligned}
$$
which shows that $\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$ is a semi-positive definite matrix.

Finally, we have shown that $f_\mu$ is $\mu$-strongly convex.


__(i)__ (1 point) Is it possible for a strongly convex function to not attain its minimum? Justify your reasoning (you may assume the domain is $\mathbb{R}^{p\times C}$).


No, it's not possible.

We assume that $f$ is a $\mu$-strongly convex function, we can get for very $\mathbf{X}\in \mathbf{R}^{p\times C}$:
$$
f(\mathbf{X})\ge f(\mathbf{O})+\langle \nabla f(\mathbf{O}), \mathbf{X} \rangle+\frac{\mu}{2}\|\mathbf{X}\|_F^2 \quad \text{where } \mathbf{O}=(0)_{p\times C}.
$$
Since $f(\mathbf{O})$ is a constant and $\langle \nabla f(\mathbf{O}), \mathbf{X} \rangle$ is a linear term, as $\|\mathbf{X}\|_F\to +\infty$, $f(\mathbf{X})\to +\infty$. Therefore, there is a threshold $M$ that for every $\mathbf{X}$ satisfying $\|\mathbf{X}\|_F>M$, $f(\mathbf{X})>f(\mathbf{O})$. At this point, we can claim that the minimum of the funtion $f$ must be attained by a specific $\mathbf{X}$ satisfying $\|\mathbf{X}\|_F\le M$.

We will now show that $f_\mu$ is smooth, i.e., $\nabla f_\mu$ is L-Lipschitz with respect to the Frobenius norm, with a simple conservative bound
$$
 L = \|\mathbf{A}\|_F^2 + \mu.
$$
where
$$
 \mathbf{A} = \begin{bmatrix}
  \leftarrow &  \mathbf{a}_1^\top & \rightarrow \\
  \leftarrow &  \mathbf{a}_2^\top & \rightarrow \\
   &  \ldots &  \\
  \leftarrow &  \mathbf{a}_n^\top & \rightarrow \\
 \end{bmatrix}.
$$
(You may use that the operator norm of the softmax Jacobian is bounded by 1/4, and a looser bound $\le 1$ is acceptable for grading.)

Hint: check the properties of the spectral norm with respect to dot product, Kronecker product, and outer product.

(1 point for all three questions)


__(j-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.


Proof:

Lets's assume $\lambda$ is an engenvalue of $\mathbf{a}_i\mathbf{a}_i^\top$ and $\mathbf{x}$ is the corresponding eigenvector. We can get: 

$$
\mathbf{a}_i\mathbf{a}_i^\top\mathbf{x}=\lambda \mathbf{x}\Rightarrow \mathbf{x}^\top\mathbf{a}_i\mathbf{a}_i^\top\mathbf{x}=\lambda \mathbf{x}^\top\mathbf{x}\Rightarrow \lambda = \frac{\|\mathbf{a}_i^\top\mathbf{x}\|^2_2}{\|\mathbf{x}\|^2_2}.
$$
So, we can use this equation to derive the largest eigenvalue of $\mathbf{a}_i\mathbf{a}_i^\top$:
$$
\begin{aligned}
\lambda_{\max}(\mathbf{a}_i\mathbf{a}^\top)
&= \max\frac{\|\mathbf{a}_i^\top\mathbf{x}\|^2_2}{\|\mathbf{x}\|^2_2} \\
&= \max_{\|\mathbf{x}\|_2=1} \|\mathbf{a}_i^\top\mathbf{x}\|^2_2 \\
&= \max_{\|\mathbf{x}\|_2=1} (\sum_{j=1}^p a_{ij}x_j)^2 \\
&\le \|\mathbf{a}_i\|^2_2\quad(\text{Cauchy's inequality}).
\end{aligned}
$$
Equality holds if and only if $\mathbf{a}_i$ and $\mathbf{x}$ are linearly dependent. Therefore, $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^\top) = \left\| \mathbf{a}_i\right\|_2^2$.

__(j-2)__ Using (2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{X})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$.

Proof:

First, we calculate the largest eigenvalue of $\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$. We assume that $\lambda$ is the eigenvalue of $\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$ and $\mathbf{x}$ is the corresponding eigenvector. According to part (h) and part (j-1), we can get:
$$
\lambda_{\max}(\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big))=\max_{\|\mathbf{x}\|=1}\operatorname{Var}(\mathbf{X})=\max_{\|\mathbf{x}\|=1}\big(\mathbb{E}[\mathbf{X}^2]-(\mathbb{E}[\mathbf{X}])^2\big)\le 1
$$
By the definition of Kronecker product, the largest eigenvalue of $(\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)$ equals $\lambda_{\max}(\mathbf{a}_i\mathbf{a}^\top)\cdot\lambda_{\max}(\big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big))$ when both matrices are semi-positive definite, which is true. Hence, $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)\le \|\mathbf{a}_i\|_2^2$. Therefore, we can show that:
$$\lambda_{\max}(\nabla^2 f_\mu(\mathbf{X})) = \lambda_{\max}(\sum_{i=1}^{n} (\mathbf{a}_i\mathbf{a}_i^\top) \otimes \big( \operatorname{Diag}(\mathbf{p}_i) - \mathbf{p}_i\mathbf{p}_i^\top \big)) + \mu \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu.$$


__(j-3)__ Conclude that $f_\mu$ is $L$-smooth for $L = \|\mathbf{A}\|_F^2 + \mu$.


Proof:

In other words, We need to prove $L-\nabla^2(f_\mu)$ is non-negative:
$$
L-\nabla^2(f_\mu)\ge\|\mathbf{A}\|_F^2-\sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2=0.
$$
Therefore, $f_\mu$ is $L$-smooth for $L = \|\mathbf{A}\|_F^2 + \mu$.

__(l)__ (1 point) KL divergence and NLL. Let $q(b_i\mid\mathbf{a}_i)$ be the true label distribution and $p(b_i\mid\mathbf{a}_i)$ the model softmax. Write the KL divergence $\mathrm{KL}(q\,\|\,p)$ and show that minimizing the KL divergence between $q$ and $p$ is equivalent to minimizing the negative log-likelihood derived in (a).


Proof:

The KL divergence between the true label distribution $q(b_i|\mathbf{a}_i)$ and the model's prediction $p(b_i|\mathbf{a}_i)$ for a single sample $i$ is:
$$
\mathrm{KL}(q\,\|\,p) = \sum_{k=1}^C q(b_i=k|\mathbf{a}_i) \log\left(\frac{q(b_i=k|\mathbf{a}_i)}{p(b_i=k|\mathbf{a}_i)}\right)
$$
The true label distribution $q$ is a one-hot vector, where $q(b_i=k|\mathbf{a}_i)=1$ for the true class $b_i$ and 0 otherwise. Due to this, the sum can be rewritten as:
$$
\mathrm{KL}(q\,\|\,p) = \log\left(\frac{1}{p(b_i=b_i|\mathbf{a}_i)}\right) = -\log p(b_i=b_i|\mathbf{a}_i).
$$
This shows that for a single sample, the KL divergence is identical to NLL.

Now, we substitute the definition of the model's softmax probability $p$ into this expression:
$$
\begin{aligned}
\mathrm{KL}(q\,\|\,p) &= -\log \left( \frac{\exp(\mathbf{a}_i^\top \mathbf{x}_{b_i})}{\sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k)} \right) \\
&= -\left( \mathbf{a}_i^\top \mathbf{x}_{b_i} - \log \sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k) \right) \\
&= -\mathbf{a}_i^\top \mathbf{x}_{b_i} + \log \sum_{k=1}^C \exp(\mathbf{a}_i^\top \mathbf{x}_k).
\end{aligned}
$$
This expression is exactly the term for sample $i$ inside the summation of the total NLL, $f(\mathbf{X})$, as derived in (a). Since this equivalence holds for every sample, the total loss function is the sum of the individual KL divergences:
$$
f(\mathbf{X}) = \sum_{i=1}^n \mathrm{KL}(q_i\,\|\,p_i).
$$
Therefore, minimizing the KL divergence is equivalent to minimizing the negative log-likelihood derived in (a).

From your work in this section, you have shown that the maximum likelihood estimator for multiclass softmax logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_F^2$ regularizer. Consequently, the estimator for $\mathbf{X}$ we will use will be the solution of the smooth strongly convex problem,
$$
 \mathbf{X}^\star = \arg\min_{\mathbf{X} \in \mathbb{R}^{p\times C}} f(\mathbf{X}) + \frac{\mu}{2}\|\mathbf{X}\|_F^2.\tag{3}
$$


## Binary logistic regression (specialization for Part 2)

While this part analyzed the multiclass (softmax) setting, in the next exercise we will continue under the simplified two-class case.

Let labels be $b_i \in \{-1, +1\}$, features $\mathbf{a}_i \in \mathbb{R}^p$, and weight vector $\mathbf{x} \in \mathbb{R}^p$. Define the sigmoid
$$
\sigma(t) = \frac{1}{1+e^{-t}}.
$$
Model the conditional distribution as
$$
\mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma\big(j\, \mathbf{a}_i^\top \mathbf{x}\big), \quad j \in \{-1,+1\}.
$$
The likelihood over i.i.d. samples $\{(\mathbf{a}_i, b_i)\}_{i=1}^n$ is
$$
\mathcal{L}(\mathbf{x}) = \prod_{i=1}^n \sigma\big(b_i\, \mathbf{a}_i^\top \mathbf{x}\big),
$$
so the negative log-likelihood is
$$
 f(\mathbf{x}) = -\log \mathcal{L}(\mathbf{x}) = \sum_{i=1}^n \log\big(1 + e^{-b_i\, \mathbf{a}_i^\top \mathbf{x}}\big).
$$

__(m)__ (1 point) Show that the gradient of the negative log-likelihood is the standard binary logistic regression gradient:
$$
\nabla f(\mathbf{x}) = \sum_{i=1}^n \big(-b_i\, \sigma(-b_i\, \mathbf{a}_i^\top \mathbf{x})\big)\, \mathbf{a}_i.
$$
(Hint: use the chain rule and $\sigma'(t) = \sigma(t)\big(1-\sigma(t)\big)$.)

We will use this binary formulation in Part 2 - First order methods.


Proof:

To show the gradient of the negative log-likelihood, we can first simply compute $\nabla \log\left(1+e^{-b_i \mathbf{a}_i^\top \mathbf{x}}\right)$:
$$
\begin{aligned}
\nabla \log\left(1+e^{-b_i \mathbf{a}_i^\top \mathbf{x}}\right)
&=\frac{e^{-b_i \mathbf{a}_i^\top \mathbf{x}}}{1+e^{-b_i \mathbf{a}_i^\top \mathbf{x}}}(-b_i\mathbf{a}_i) \\
&=\left(-b_i\frac{1}{1+e^{b_i \mathbf{a}_i^\top \mathbf{x}}}\right)\mathbf{a}_i \\
&=\left(-b_i\sigma(-b_i \mathbf{a}_i^\top x)\right)\mathbf{a}_i.
\end{aligned}
$$
Therefore, $\nabla f(\mathbf{x}) = \sum_{i=1}^n \big(-b_i\, \sigma(-b_i\, \mathbf{a}_i^\top \mathbf{x})\big)\, \mathbf{a}_i$.